Demo of llama3 tool selections with OCI Generativie AI Inference  

In [ ]:
#Variables
COMPARTMENT_ID = "COMPARTMENT OCID"
AUTH_TYPE = "API_KEY" # The authentication type to use, e.g., API_KEY (default), SECURITY_TOKEN, INSTANCE_PRINCIPAL, RESOURCE_PRINCIPAL.
CONFIG_PROFILE = "OCI CONFIG PROFILE NAME"
MODEL_ID="meta.llama-3.3-70b-instruct"

In [2]:
#Define controller
import oci
config = oci.config.from_file(profile_name=CONFIG_PROFILE)
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config)
generative_ai_client = oci.generative_ai.GenerativeAiClient(config)


In [3]:
#Tools
timesheet_tool = oci.generative_ai_inference.models.FunctionDefinition(
                    type="FUNCTION",
                    name="timesheet_tool",
                    description="tool to use for any queries relatead to timesheet",
                    parameters='{"a": "int"}')
leave_tool = oci.generative_ai_inference.models.FunctionDefinition(
                    type="FUNCTION",
                    name="leave_tool",
                    description="tool for any leave related queries",
                    parameters='{"a": "int"}')
generic_tool = oci.generative_ai_inference.models.FunctionDefinition(
                    type="FUNCTION",
                    name="generic_tool",
                    description="tool for any queries that does not use any of the other listed tools",
                    parameters='{"a": "int"}')

In [ ]:
import os #Optional to disable proxy
os.environ['no_proxy'] = '*' 

In [7]:
def call_inference(prompt:str=None):
    chat_response = generative_ai_inference_client.chat(
        chat_details=oci.generative_ai_inference.models.ChatDetails(
            compartment_id=COMPARTMENT_ID,
            serving_mode=oci.generative_ai_inference.models.OnDemandServingMode(
                serving_type="ON_DEMAND",
                model_id=MODEL_ID),
            chat_request=oci.generative_ai_inference.models.GenericChatRequest(
                api_format="GENERIC",
                messages=[
                    oci.generative_ai_inference.models.UserMessage(
                        role="SYSTEM",
                        content=[
                            oci.generative_ai_inference.models.TextContent(
                                type="TEXT",
                                text="Be a helpful agent")],),
                    oci.generative_ai_inference.models.UserMessage(
                        role="USER",
                        content=[
                            oci.generative_ai_inference.models.TextContent(
                                type="TEXT",
                                text=prompt),
                                    ],
                        name="user_message")],
                is_stream=False,
                num_generations=1,
                is_echo=True,
                top_k=-1,
                top_p=0.22750199,
                temperature=0,
                frequency_penalty=1.7431623,
                presence_penalty=1.7700758,
                max_tokens=423,
                tool_choice=oci.generative_ai_inference.models.ToolChoiceRequired(
                    type="AUTO"),
                tools=[timesheet_tool,leave_tool,generic_tool])),)
    return chat_response
    
    

In [8]:
prompts = [
    "how many leaves i have left with ?",
    "how many leaves i can enter via timesheet",
    "how many employees are in this org?",
    "how many timesheet entriees i can make while i am on sebatical leave?",
    "what is the minimum tenure requirement for internal job switch"
]

In [9]:

for prompt in prompts:   
    response = call_inference(prompt=prompt)
    if response.status == 200:
        print(f"✅️ Prompt : {prompt},Tool: {response.data.chat_response.choices[0].message.tool_calls[0].name}")
    else:
        print(f"❌ Failed to run - {response.status}")


✅️ Prompt : how many leaves i have left with ?,Tool: leave_tool
✅️ Prompt : how many leaves i can enter via timesheet,Tool: timesheet_tool
✅️ Prompt : how many employees are in this org?,Tool: generic_tool
✅️ Prompt : how many timesheet entriees i can make while i am on sebatical leave?,Tool: timesheet_tool
✅️ Prompt : what is the minimum tenure requirement for internal job switch,Tool: generic_tool
